In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
import os
import numpy as np
import xml.etree.ElementTree as ET
import functools
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import LearningRateScheduler


strategy = tf.distribute.MirroredStrategy()

# 라벨
def read_xml_file(xml_file_path):
    if not os.path.isfile(xml_file_path):
        return []
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        labels.append(label)
    return labels

def read_xml_and_add_labels(image_path, label_directory):
    if not os.path.isfile(image_path):
        return []
    image_name = os.path.basename(image_path)
    class_name = os.path.dirname(image_path).split(os.path.sep)[-1]
    xml_file_path = os.path.join(label_directory, class_name, image_name[:-4] + '.xml')
    labels = read_xml_file(xml_file_path)
    return labels

input_shape = (256, 256, 3)

#매번바꿔주기
num_classes = 205
batch_size = 128

initial_learning_rate = 0.048
momentum = 0.9
weight_decay = 0.00004
dropout_rate = 0.2
num_epochs = 50

with strategy.scope():

    # 데이터증강기법
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=lambda img: tf.image.random_crop(img, [256, 256, 3])
    )

    # 훈련 데이터셋
    train_directory = 'E:/종설/Training/1/[원천]면류'
    train_label_directory = 'E:/종설/Training/2'
    train_dataset = train_datagen.flow_from_directory(
        train_directory,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # 훈련라벨
    train_labels = []
    for i, image_path in enumerate(train_dataset.filepaths):
        labels = read_xml_and_add_labels(image_path, train_label_directory)
        train_labels.append(labels)


    train_labels = np.array(train_labels)

    # 크기조정만
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255
    )

    # 검증데이터
    val_directory = 'E:/종설/Validation/1/[원천]면류'
    val_label_directory = 'E:/종설/Validation/2'
    val_dataset = val_datagen.flow_from_directory(
        val_directory,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # 검증라벨
    val_labels = []
    for i, image_path in enumerate(val_dataset.filepaths):
        labels = read_xml_and_add_labels(image_path, val_label_directory)
        val_labels.append(labels)

    val_labels = np.array(val_labels)

    # EfficientNet-B0모델 불러오기
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
        
    # 층 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    def lr_schedule(epoch):
        initial_lr = initial_learning_rate
        decay_factor = 0.97
        decay_epochs = 2.4
        lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
        return lr

    # Define the optimizer with the learning rate schedule
    optimizer = Adam(learning_rate=lr_schedule(0), beta_1=momentum, decay=weight_decay)

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    lr_scheduler = LearningRateScheduler(lr_schedule)

    history = model.fit(
        train_dataset,
        steps_per_epoch=len(train_dataset),
        epochs=num_epochs,
        callbacks=[lr_scheduler],
        validation_data=val_dataset,
    )

    # Save the model
    best_val_acc = max(history.history['val_accuracy'])
    model.save('my_model_noodle_v4.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Found 23370 images belonging to 205 classes.


C:\Users\sj990\AppData\Local\Temp\ipykernel_2528\3239692883.py:78: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_labels = np.array(train_labels)


Found 3075 images belonging to 205 classes.


C:\Users\sj990\AppData\Local\Temp\ipykernel_2528\3239692883.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val_labels = np.array(val_labels)


Epoch 1/50
183/183 [==============================] - 2288s 12s/step - loss: 5.6873 - accuracy: 0.0046 - val_loss: 5.3325 - val_accuracy: 0.0049
Epoch 2/50
183/183 [==============================] - 2425s 13s/step - loss: 5.3397 - accuracy: 0.0041 - val_loss: 5.3331 - val_accuracy: 0.0049
Epoch 3/50
 25/183 [===>..........................] - ETA: 25:06 - loss: 5.3376 - accuracy: 0.0060

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
import os
import numpy as np
import xml.etree.ElementTree as ET
import functools
from tensorflow.keras.optimizers import RMSprop, Adam


strategy = tf.distribute.MirroredStrategy()

# 라벨
def read_xml_file(xml_file_path):
    if not os.path.isfile(xml_file_path):
        return []
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        labels.append(label)
    return labels

def read_xml_and_add_labels(image_path, label_directory):
    if not os.path.isfile(image_path):
        return []
    image_name = os.path.basename(image_path)
    class_name = os.path.dirname(image_path).split(os.path.sep)[-1]
    xml_file_path = os.path.join(label_directory, class_name, image_name[:-4] + '.xml')
    labels = read_xml_file(xml_file_path)
    labels = np.array(labels, dtype=np.int32)  # Convert to numpy array
    return labels


with strategy.scope():
    # 데이터증강기법
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        preprocessing_function=lambda img: tf.image.random_crop(img, [256, 256, 3])
    )

    train_directory = 'E:/종설/Training/1/[원천]면류'
    train_label_directory = 'E:/종설/Training/2'

    train_dataset = train_datagen.flow_from_directory(
        train_directory,
        target_size=(256, 256),
        batch_size=120,
        shuffle=True,
        seed=42,
        follow_links=True,
        classes=None,
        class_mode=None  # Set class_mode to None to return only the images without labels
    )

    train_filepaths = train_dataset.filepaths
    train_labels = [read_xml_and_add_labels(image_path, train_label_directory) for image_path in train_filepaths]

    # Find the maximum length of sequences in train_labels
    max_length = max(len(labels) for labels in train_labels)

    # Pad sequences in train_labels to the maximum length
    train_labels = [labels + [0] * (max_length - len(labels)) for labels in train_labels]

    train_dataset = tf.data.Dataset.from_tensor_slices((train_filepaths, train_labels))

    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )

    val_directory = 'E:/종설/Validation/1/[원천]면류'
    val_label_directory = 'E:/종설/Validation/2'
    val_dataset = val_datagen.flow_from_directory(
        val_directory,
        target_size=(256, 256),
        batch_size=120,
        shuffle=False,
        classes=None,
        class_mode=None  # Set class_mode to None to return only the images without labels
    )

    val_filepaths = val_dataset.filepaths
    val_labels = [read_xml_and_add_labels(image_path, val_label_directory) for image_path in val_filepaths]

    # Pad sequences in val_labels to the maximum length
    val_labels = [labels + [0] * (max_length - len(labels)) for labels in val_labels]

    val_dataset = tf.data.Dataset.from_tensor_slices((val_filepaths, val_labels))

    
    
    # EfficientNet-B0모델 불러오기
    pretrained_model = hub.KerasLayer("C:/Users/sj990/MachineLearning/efficientnet_b0_feature-vector_1", trainable=False)

    # Add Reshape layer to match input dimensions of GlobalAveragePooling2D
    reshape_layer = tf.keras.layers.Reshape((1, 1, 1280))

    # Add layers to the model
    model = Sequential()
    model.add(pretrained_model)
    model.add(reshape_layer)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(train_dataset.num_classes, activation='softmax', kernel_regularizer=l2(0.0005)))


    def lr_schedule(epoch):
        initial_lr = 0.0005
        decay_factor = 0.97
        decay_epochs = 2.4
        lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
        return lr

    # Define the optimizer with the learning rate schedule
    optimizer = Adam(learning_rate=lr_schedule(0), beta_1=0.9, decay=0.0005)

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    num_epochs = 50
    best_val_acc = 0.0

    history = model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=val_dataset,
    )

    # Save the model
    best_val_acc = max(history.history['val_accuracy'])
    model.save('my_model_noodle_v4.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Found 23370 images belonging to 205 classes.


ValueError: invalid literal for int() with base 10: '농심오징어짬뽕컵67G'

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
import os
import numpy as np
import xml.etree.ElementTree as ET
import functools
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import LearningRateScheduler
import pandas as pd

strategy = tf.distribute.MirroredStrategy()

input_shape = (256, 256, 3)

#매번바꿔주기
num_classes = 205
batch_size = 128

initial_learning_rate = 0.048
momentum = 0.9
weight_decay = 0.00004
dropout_rate = 0.2
num_epochs = 50

with strategy.scope():
    
    # 훈련 데이터셋
    train_directory = 'E:/종설/Training/1/[원천]면류'
    train_label_directory = 'E:/종설/Training/2'

    # 데이터증강기법
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=lambda img: tf.image.random_crop(img, [256, 256, 3])
    )

    train_dataset = train_datagen.flow_from_directory(
        train_directory,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
     # 검증데이터
    val_directory = 'E:/종설/Validation/1/[원천]면류'
    val_label_directory = 'E:/종설/Validation/2'
    
    # 크기조정만
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255
    )

    
    val_dataset = val_datagen.flow_from_directory(
        val_directory,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical'
    )
           
    # EfficientNet-B0모델 불러오기
    pretrained_model = hub.KerasLayer("C:/Users/sj990/MachineLearning/efficientnet_b0_feature-vector_1")

    # 층 추가
    model = Sequential()
    model.add(pretrained_model)
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(train_dataset.num_classes, activation='softmax', kernel_regularizer=l2(0.0005)))

    # 동결
    for layer in model.layers[:-1]:
        layer.trainable = False

    def lr_schedule(epoch):
        initial_lr = initial_learning_rate
        decay_factor = 0.97
        decay_epochs = 2.4
        lr = initial_lr * (decay_factor ** (epoch // decay_epochs))
        return lr

    # Define the optimizer with the learning rate schedule
    optimizer = Adam(learning_rate=lr_schedule(0), beta_1=momentum, decay=weight_decay)
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    lr_scheduler = LearningRateScheduler(lr_schedule)

    history = model.fit(
        train_dataset,
        steps_per_epoch=len(train_dataset),
        epochs=num_epochs,
        callbacks=[lr_scheduler],
        validation_data=val_dataset, 
        validation_steps=len(val_dataset)
    )

    # Save the model
    best_val_acc = max(history.history['val_accuracy'])
    model.save('my_model_noodle_v4.h5') 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Found 23370 images belonging to 205 classes.
Found 3075 images belonging to 205 classes.
Epoch 1/50
 43/183 [======>.......................] - ETA: 25:05 - loss: 5.4408 - accuracy: 0.2470

In [46]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
        
# 1. 모델 불러오기
model = tf.keras.models.load_model('my_model_noodle_v2.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 2. 이미지 불러오기(jpg or png파일형식)
img_path = r'C:\Users\sj990\Desktop\ex\shin.jpg'

# 이미지전처리, 사이즈조정만
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
img = tf.keras.preprocessing.image.img_to_array(img)
img = img.astype('float32') / 255.0
img = np.expand_dims(img, axis=0)

# 이미지 예측
preds = model.predict(img)

class_labels = []

# 3. txt파일 불러오기
with open('class_labels_noodle.txt', 'r') as file:
    for line in file:
        class_labels.append(line.strip())

# 결과값 상위 3개
top_preds_idx = preds[0].argsort()[ : : -1][ : 1]
top_preds_labels = [class_labels[idx] for idx in top_preds_idx]
top_preds_probs = preds[0][top_preds_idx]

results = []

#json형식 results
for label, prob in zip(top_preds_labels, top_preds_probs):
    result = {
        '상품': label,
        '정확도': round(float(prob), 2)
    }
    results.append(result)
        

#json파일 저장필요할시 사용
with open('results.json', 'w', encoding='utf-8') as file:
    json.dump(results, file, ensure_ascii=False)


#출력예시
print(results)


[{'상품': '35161_농심)신라면큰사발면114G', '정확도': 0.98}]
